In [1]:
import pandas as pd
from pathlib import Path

In [2]:
sub_folder = "13"

results_path = Path("results") / sub_folder # Directory where we will store all the results
results_path.mkdir(exist_ok=True, parents=True)

script_base_path = Path("scripts") / sub_folder # Path to directory with the chapter CSVs
script_file_names = [f"chapter{i}.csv" for i in range(1, 7)] # "chapter1.csv" - "chapter6.csv"

df = pd.concat([pd.read_csv(script_base_path / path) for path in script_file_names]).dropna(how="all").drop_duplicates("Filename")
df.describe()

,Filename,Voice,Line,Notes
count,347,321,347,13
unique,347,35,347,12
top,Script_4_Communication_13,Slacks,Calm down friend. Now go pick up that odd blue...,delet dis
freq,1,129,1,2


In [3]:
df.head()

,Filename,Voice,Line,Notes
0,Script_1_Opening_1,Slacks,Oh look at this! New blood! The dragon knight ...,NaN
1,Script_1_Opening_2,SUNSfan,Fresh from the Netflix anime are you? Welcome ...,NaN
2,Script_1_Opening_3,Slacks,Except for Artifact.,alt-and the most confusing!
3,Script_1_Opening_4,SUNSfan,"Yes, yes, of course, except for Artifact.",alt-yes its quite confusing
4,General_Boo,Indiegogo,*Boo*,NaN


In [4]:
# Write addon_english lines
localizations_path = results_path / "addon_english.txt"

with open(localizations_path, "w", encoding="utf-8") as loc_file:
    for row in df[["Filename", "Line"]].to_numpy():
        try:
            cleaned_line = row[1].strip().replace("\"", "'")
            loc_file.write(f'        "{row[0]}" "{cleaned_line}"\n')
        except:
            print("Error on line", row)

In [5]:
# Write localization key typescript object
enum_path = results_path / "localization.ts"

with open(enum_path, "w", encoding="utf-8") as enum_file:
    for row in df[["Filename", "Line"]].to_numpy():
        enum_file.write(f'    {row[0]} = "{row[0]}",\n')

In [6]:
# Write sound events file
sound_events_path = results_path / "tutorial_dialogs.vsndevts"

sound_template = """    "%s" =
    {
        type = "dota_src1_2d"
        volume = "1.0"
        vsnd_files = [ "sounds/tutorial_dialogs/%s.vsnd" ]
    }
""".replace("\r\n", "\n")

with open(sound_events_path, "w", encoding="utf-8") as sound_file:
    sound_file.write("<!-- kv3 encoding:text:version{e21c7f3c-8a33-41c5-9977-a76d3a32aa0d} format:generic:version{7412167c-06e9-4698-aff2-e63eb59037e7} -->\n")
    sound_file.write("{\n")
    for row in df["Filename"].to_numpy():
        sound_file.write(sound_template % (row, row))
    sound_file.write("}\n")

In [7]:
# Write mp3 lengths typescript object (requires mutagen)
from mutagen.mp3 import MP3

audio_base_path = Path("..") / "content" / "sounds" / "tutorial_dialogs" # Path where the mp3s are
sound_lengths_path = results_path / "Sounds.ts"

with open(sound_lengths_path, "w", encoding="utf-8") as sound_file:
    sound_file.write("export const soundDurations: Record<string, number> = {\n")
    for row in df["Filename"].to_numpy():
        path = audio_base_path / f"{row}.mp3"
        if (path.exists()):
            duration = MP3(path).info.length
            sound_file.write('    "%s": %f,\n' % (row, duration))
    sound_file.write("}\n")